In [73]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import SnowballStemmer
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
from csv_helper import CSVHelper
import numpy as np
import collections
import re

[nltk_data] Downloading package stopwords to /home/jeroen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jeroen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [74]:
data = CSVHelper.load_csv("Tweets_2016London.csv")


starting for loop!


In [75]:
def tokenize(tweets):
    tknzr = TweetTokenizer()
    tokenized_tweets = []
    for tweet in tweets:
        tokenized_tweets.append(tknzr.tokenize(tweet))
    return tokenized_tweets

def remove_stopwords(tweets):
    stopwords =  nltk.corpus.stopwords.words('english')
    tweets_nostop = []
    for tweet in tweets:
        tweet_nostop = [w.lower() for w in tweet if w.lower() not in stopwords]
        tweets_nostop.append(tweet_nostop)
    return tweets_nostop

def filter_noise(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub("[^a-zA-Z0-9]",'', w) for w in tweet if re.sub("[^a-zA-Z0-9]",'', w)!= '']
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

def remove_url(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub(r'^https?:\/\/.*[\r\n]*', '', w) for w in tweet]
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

#we decided to use a lemmatizer instead of stemming 
def my_stem(tweets):
    lmt = WordNetLemmatizer()
    ps = PorterStemmer()
    tweets_stemmed = []
    for tweet in tweets:
        tweet_stemmed = [lmt.lemmatize(w) for w in tweet]
        tweets_stemmed.append(tweet_stemmed)
    return tweets_stemmed

In [76]:
data = tokenize(data)
data = remove_url(data)
data = filter_noise(data)
data = remove_stopwords(data)
data = my_stem(data)
print(data)

[['please', 'see', 'link', 'youtube', 'channel', 'darteh', 'ministry', 'please', 'like', 'share', 'subscribe'], ['northeastedl', 'lol', 'happens', 'put', 'woman', 'charge'], ['side', 'man', 'get', 'smash', 'show', 'shout', 'tv', 'swear', 'get', 'ladddd'], ['le', '1', 'month'], ['dunzocs', 'lived', '3', 'year', 'visit', 'la', 'every', 'month', 'home', 'second', 'home', 'tbh', 'bday', 'could'], ['described', 'met', 'heart', 'broken', 'im', 'touched'], ['thetwomikes', 'dcrosss', 'talksport', 'talkradio', 'ya', 'dozy', 'sausage'], ['sometimes', 'life', 'hard', 'dust', 'keep', 'going', 'nolimit'], ['dunzocs', 'end', 'happening'], ['tmp', '03', 'c', 'wind', '0mph', 'press', '10274', 'mb', 'cloud', '837', 'ft', 'rain', '00', 'mm', 'humidity', '86', 'fine', 'weather', 'ukweather'], ['themissing', 'cmaawards50', 'wembley', 'spur', 'thistimenextyear', '201611', '3', '0114', 'gmt', 'trndnl'], ['tgfone4', 'worldseries', 'real', 'madrid', 'andy', 'carroll', 'sissoko', '201611', '3', '0114', 'gmt', 

In [80]:
def termfreq(term, doc):
    length = len(doc)
    frequency = doc.count(term)
    return frequency/length


def count_doc_with_term(term, tweets):
    ctr = 0
    for tweet in tweets:
        if term in tweet:
            ctr = ctr + 1
    return ctr

def idf(term, tweets):
    N = len(tweets)
    return np.log(N/count_doc_with_term(term, tweets))



vocabulary = set()
for tweet in data:
    words = [w for w in tweet]
    vocabulary.update(words)

vocabulary = list(vocabulary)
word_idx = {w: idx for idx, w in enumerate(vocabulary)}

VOCABULARY_SIZE = len(vocabulary)
DOCUMENTS_COUNT = len(data)

word_idf = collections.defaultdict(lambda: 0)
for tweet in data:
    words = set(tweet)
    for w in words:
        word_idf[w] +=1
        
for w in vocabulary:
    word_idf[w] = idf(w, data)
    

def tf_idf(term, doc, tweets):
    return termfreq(term, doc) * word_idf[term]

tf_idf('youtube', data[0] ,data)

      

defaultdict(<function <lambda> at 0x7f2aa3479378>, {'like': 71, 'please': 25, 'darteh': 1, 'ministry': 1, 'share': 2, 'youtube': 42, 'channel': 40, 'subscribe': 40, 'see': 60, 'link': 4, 'lol': 11, 'northeastedl': 1, 'woman': 13, 'charge': 1, 'put': 10, 'happens': 2, 'shout': 3, 'smash': 2, 'get': 52, 'tv': 5, 'show': 21, 'man': 22, 'side': 8, 'swear': 2, 'ladddd': 1, 'month': 12, '1': 18, 'le': 2, 'every': 16, 'year': 45, 'dunzocs': 2, 'la': 1, 'tbh': 1, 'lived': 2, '3': 26, 'second': 4, 'could': 11, 'home': 12, 'bday': 2, 'visit': 9, 'broken': 2, 'heart': 2, 'touched': 1, 'met': 4, 'described': 1, 'im': 85, 'dcrosss': 1, 'talksport': 2, 'thetwomikes': 1, 'ya': 7, 'dozy': 1, 'talkradio': 1, 'sausage': 1, 'life': 24, 'going': 31, 'keep': 9, 'hard': 8, 'sometimes': 4, 'nolimit': 1, 'dust': 1, 'happening': 2, 'end': 17, 'c': 25, 'ft': 11, 'humidity': 12, 'press': 13, 'mb': 12, '86': 3, 'tmp': 6, 'wind': 18, '03': 8, 'cloud': 9, 'weather': 17, '10274': 1, 'ukweather': 9, 'mm': 18, 'fine':

0.35124842875457862

In [ ]:
print(word_idf['watch'])

In [ ]:
http://billchambers.me/tutorials/2014/12/21/tf-idf-explained-in-python.html